In [86]:
from keras.models import load_model
import numpy as np
import sys
import os
import pickle as pk
import pandas as pd
from os import listdir
from os.path import isfile, join

from keras.utils import to_categorical ,Sequence
from keras import losses, models, optimizers
from keras.activations import relu, softmax
from keras.callbacks import (EarlyStopping, LearningRateScheduler,
                             ModelCheckpoint, TensorBoard, ReduceLROnPlateau)

from keras.layers import (Convolution1D, Dense, Dropout, GlobalAveragePooling1D, 
                          GlobalMaxPool1D, Input, MaxPool1D, concatenate)

from keras.layers import (Convolution2D, GlobalAveragePooling2D, BatchNormalization, Flatten,
                          GlobalMaxPool2D, MaxPool2D, concatenate, Activation)

from keras import backend as K                      


# calculate accuracy
from sklearn.metrics import accuracy_score

In [87]:
map_dict = pk.load(open('data/map.pkl' , 'rb'))
df = pd.read_csv('public_csv/ans_0.894.csv')

In [88]:
df.head(3)
df.shape

(9400, 2)

In [89]:
fname = df['fname'].tolist()
label = df['label'].tolist()
fname


['00063640.wav',
 '0013a1db.wav',
 '002bb878.wav',
 '002d392d.wav',
 '00326aa9.wav',
 '0038a046.wav',
 '003995fa.wav',
 '005ae625.wav',
 '007759c4.wav',
 '008afd93.wav',
 '00a161c0.wav',
 '00a7a2f6.wav',
 '00ae03f6.wav',
 '00b2404e.wav',
 '00beb030.wav',
 '00c4d5b8.wav',
 '00c92c05.wav',
 '00ccf065.wav',
 '00d0ab77.wav',
 '00dffe3a.wav',
 '00e33205.wav',
 '00e7ed07.wav',
 '00eac343.wav',
 '00f2f692.wav',
 '0102a895.wav',
 '010a0b3a.wav',
 '010eed01.wav',
 '01115622.wav',
 '01198b7e.wav',
 '01207ce5.wav',
 '0137bfba.wav',
 '014eb52e.wav',
 '01608fa3.wav',
 '0169348e.wav',
 '016e52da.wav',
 '0183b482.wav',
 '0185de6b.wav',
 '01a5a2a3.wav',
 '01aed32a.wav',
 '01afccc5.wav',
 '01b4cdb5.wav',
 '01b85217.wav',
 '01bb344f.wav',
 '01bede12.wav',
 '01c9d6d3.wav',
 '01e628cb.wav',
 '01e6e112.wav',
 '01e99e2d.wav',
 '01eccb6e.wav',
 '01f27f99.wav',
 '01f29edb.wav',
 '01f9883e.wav',
 '01fd22ba.wav',
 '01fe4a01.wav',
 '02107093.wav',
 '02182ebd.wav',
 '02198549.wav',
 '0220d48a.wav',
 '02210ee7.wav

# ensemble for public_csv

In [90]:

folder_path = 'public_csv'
csv_files = [join(folder_path, f) for f in listdir(folder_path) if isfile(join(folder_path, f))]

result = np.zeros((df.shape[0] , 41))

for p in csv_files:
    df = pd.read_csv(p)
    label = df['label'].tolist()
    predict = [x.split() for x in label]
    
    # convert to label
    for i,d in enumerate(predict):
        a = []
        for label in d:
            a.append(map_dict[label])
            
        predict[i] = a
    
    
    # convert to one-hot
    one_hot = np.zeros((df.shape[0] , 41))
    for i , d in enumerate( predict):

        for top ,label in enumerate( d ) :
            # 0 -> 1-(0 * 0.2) = 1
            # 1 -> 1-(1* 0.2) = 0.8
            # 2 -> 1-(2 *0.2) = 0.6

            weight = 1 - (top * 0.2)
            one_hot[i][label] = weight
    
    result += one_hot

# Create ans.csv

In [91]:
output = []
reverse_dict = pk.load(open('data/map_reverse.pkl' , 'rb'))

for i , d in enumerate( result):

    top3 = d.argsort()[-3:][::-1]
    result = [reverse_dict[x] for x in top3]
    s = ' '.join(result)
    output.append(s)
    
df = pd.DataFrame(output , columns = ['label'])
df.insert(0,'fname' , fname)
df.to_csv('public_voting.csv', index=False)
print(df.head(3))

          fname                               label
0  00063640.wav         Shatter Tambourine Scissors
1  0013a1db.wav         Flute Violin_or_fiddle Meow
2  002bb878.wav  Bass_drum Gunshot_or_gunfire Knock
